In [1]:
import itertools
import operator
import gensim
import konlpy
import pandas as pd
import scipy as sp 
from konlpy.tag import Kkma
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import pairwise_distances

In [2]:
tb_df = pd.read_excel('tb_des_df.xlsx')

In [4]:
tb_users = pd.read_excel('tb_users_0410.xlsx')

In [14]:
# model_word (noun base) load
model_word = gensim.models.Doc2Vec.load('model_word.model')

In [15]:
# model_sentence (sentence base) load
model_sentence = gensim.models.Doc2Vec.load('model_sentence.model')

In [158]:
# model_doc load
model_doc = gensim.models.Doc2Vec.load('doc_vec.model')

In [36]:
""" euclidean distance 구하기 """
def dist_cal(doc_model):
    import scipy as sp 
    #vector bag 만들기
    doc_vecs = []
    dists = []
    id_list = []
    eu_dist = sp.spatial.distance.euclidean
    for i in tb_df['id']:
        vec = doc_model.infer_vector('{number}'.format(number=i))
        id_list.append(i)
        doc_vecs.append(vec)
    # 거리 계산
    for a, b in itertools.combinations(np.arange(len(doc_vecs)), 2):
        dist = eu_dist(doc_vecs[a], doc_vecs[b])
        dists.append((id_list[a], id_list[b], dist))
    return dists

In [135]:
"""recommend list 출력"""
def project_recommend(dist_df, number):
    import pandas as pd
    recommend_list = pd.DataFrame()
    if number <= 2000:
        recommend = dist_df.loc[dist_df[0] == number]
        sorting = recommend.sort_values(by=[2], ascending = False)[:11]
        sorting.index = np.arange(len(sorting))
        sorting_df = sorting.drop([0], axis=1)
        for i in sorting_df[1]:
            pj = tb_df.loc[tb_df['id'] == i]
            recommend_list = recommend_list.append(pj)
        recommend_list.index = np.arange(len(recommend_list))
        return recommend_list

    else:
        recommend = dist_df.loc[dist_df[1] == number]
        sorting = recommend.sort_values(by=[2], ascending = False)[:11]
        sorting.index = np.arange(len(sorting))
        sorting_df = sorting.drop([1], axis=1)
        for i in sorting_df[0]:
            pj = tb_df.loc[tb_df['id'] == i]
            recommend_list = recommend_list.append(pj)
        recommend_list.index = np.arange(len(recommend_list))
        return recommend_list

In [39]:
# distance 계산
import time
start = time.clock()

sentence_df = pd.DataFrame(dist_cal(model_sentence))

end = time.clock()
print ("%.2gs" % (end-start))

93s


In [40]:
import time
start = time.clock()

word_df = pd.Dat(dist_cal(model_word))

end = time.clock()
print ("%.2gs" % (end-start))

90s


In [161]:
import time
start = time.clock()

doc_df = pd.DataFrame(dist_cal(model_doc))

end = time.clock()
print ("%.2gs" % (end-start))

1.2e+02s


In [173]:
analyze = CountVectorizer(analyzer="word").build_analyzer()
sentences = []
for i in des_df['description']:
    words = analyze(i)
    sentences.append(words)
text_all=[]
for i in np.arange(len(sentences)):
    text = gensim.models.doc2vec.LabeledSentence(words = sentences[i], tags = ['doc_{number}'.format(number=i)])
    text_all.append(text)

In [180]:
test = text_all[0][0]

In [ ]:
sentence_df = pd.DataFrame(model_sentence_dist)
word_df = pd.DataFrame(model_word_dist)
doc_df = pd.DataFrame(model_doc_df)

In [172]:
model_sentence_vec = model_sentence.infer_vector('12')
model_sentence.most_similar([model_sentence_vec])

[('드리겠습니다', 0.6059895157814026),
 (':-) * 감사의 마음을 담아 에필로그에 성함을 기재하고, 소중히 기억하겠습니다.', 0.4839726388454437),
 ('드리도록 하겠습니다.', 0.4824797213077545),
 ('- 3개월에 한번 씩 발행될 “ 룩” 의 다음 호를 즐겁게 기대할 수 있도록\xa0 달력 엽서를 보내',
  0.42651718854904175),
 ('잘 부탁 드립니다.', 0.4065120816230774),
 ('믿어 의심치 않습니다.', 0.38742968440055847),
 ('\xa0 \xa0 \xa0', 0.36092567443847656),
 ("1호 'UNIFORM' 을 드립니다.", 0.3489863872528076),
 ('감사합니다', 0.34610283374786377),
 ('그렇습니다.', 0.33533862233161926)]

In [181]:
model_word_vec = model_word.infer_vector(test)
model_word.most_similar([model_word_vec])

[('김건희', 0.6032652854919434),
 ('폴라로이드', 0.5642544031143188),
 ('영성', 0.5179054141044617),
 ('쇼핑', 0.49117138981819153),
 ('건희', 0.486127108335495),
 ('1996년', 0.4750169813632965),
 ('수직적', 0.4695546329021454),
 ('잠실', 0.4674193561077118),
 ('티켓값', 0.4670153260231018),
 ('능가', 0.4648624062538147)]

In [188]:
model_word_vec = model_word.infer_vector('doc_0')
model_word.most_similar([model_word_vec])

[('데코', 0.686865508556366),
 ('0호', 0.564838171005249),
 ('2000만원', 0.5607826709747314),
 ('약간의', 0.5553574562072754),
 ('코엑스', 0.5336793065071106),
 ('프란시스코', 0.5235170125961304),
 ('문화체육관광부', 0.5205384492874146),
 ('12', 0.5178917646408081),
 ('코코', 0.5138304829597473),
 ('18,000', 0.513398289680481)]

In [185]:
model_doc_vec = model_doc.infer_vector('12')
model_doc.most_similar([model_doc_vec])

[('단가는', 0.20936812460422516),
 ('분도', 0.2060094028711319),
 ('완료한', 0.19825929403305054),
 ('디자인했습니다', 0.19739557802677155),
 ('창작자간의', 0.1958780735731125),
 ('공모전', 0.19517935812473297),
 ('약간의', 0.19355371594429016),
 ('환상의', 0.19325032830238342),
 ('않았어요', 0.1880558729171753),
 ('튼튼하게', 0.1872541457414627)]

In [141]:
tb_df.ix[2000]['id']

3282

In [151]:
project_recommend(word_df, 4177)['category']

0          극영화
1           웹툰
2           게임
3     공개 소프트웨어
4           사진
5           웹툰
6         공공예술
7           연극
8         혼합매체
9           요리
10        저널리즘
Name: category, dtype: object

In [154]:
tb_df.ix[2500]['id'], tb_df.ix[2500]['category']

(4177, '출판')

In [156]:
project_recommend(sentence_df, 4177)['category']

0     일러스트레이션
1       비디오게임
2         논픽션
3       다큐멘터리
4          연극
5         디자인
6          음악
7          사진
8          음악
9          웹툰
10         웹툰
Name: category, dtype: object